<a href="https://colab.research.google.com/github/youngwoo3283/project2-section02/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import files
files.upload();

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data_game = pd.read_csv('games.csv')

# 데이터 준비하기

In [39]:
data_game.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18


In [40]:
data_champ = pd.read_json('/content/drive/MyDrive/archive (2)/champion_info.json')

In [41]:
data_champ2 = pd.read_json('/content/drive/MyDrive/archive (2)/champion_info_2.json')

In [42]:
data_spell = pd.read_json('/content/drive/MyDrive/archive (2)/summoner_spell_info.json')

In [43]:
champInfo = pd.read_json((data_champ['data']).to_json(), orient='index') 

In [44]:
champInfo2 = pd.read_json((data_champ2["data"]).to_json(),orient = "index")

In [45]:

data_spell_info = pd.read_json((data_spell['data']).to_json(), orient='index')

data_game2 = pd.read_csv('games.csv')

winner = data_game2[["winner"]]
winner = winner.T
winner.shape

(1, 51490)

In [46]:
name_dict = pd.Series(champInfo.key.values,index=champInfo.id).to_dict()

In [47]:
champInfo2

,tags,title,id,key,name
Aatrox,"[Fighter, Tank]",the Darkin Blade,266,Aatrox,Aatrox
Ahri,"[Mage, Assassin]",the Nine-Tailed Fox,103,Ahri,Ahri
Akali,[Assassin],the Fist of Shadow,84,Akali,Akali
Alistar,"[Tank, Support]",the Minotaur,12,Alistar,Alistar
Amumu,"[Tank, Mage]",the Sad Mummy,32,Amumu,Amumu
...,...,...,...,...,...
Zac,"[Tank, Fighter]",the Secret Weapon,154,Zac,Zac
Zed,"[Assassin, Fighter]",the Master of Shadows,238,Zed,Zed
Ziggs,[Mage],the Hexplosives Expert,115,Ziggs,Ziggs
Zilean,"[Support, Mage]",the Chronokeeper,26,Zilean,Zilean


## 데이터 전처리하기

## 선택한 챔피언과 금지한 챔피언 정보는 따로 저장한다

In [48]:
champs = data_game[["t1_champ1id","t1_champ2id",
                      "t1_champ3id","t1_champ4id","t1_champ5id","t2_champ1id",
                      "t2_champ2id","t2_champ3id","t2_champ4id","t2_champ5id","t1_ban1","t1_ban2","t1_ban3",
                      "t1_ban4","t1_ban5","t2_ban1","t2_ban2","t2_ban3","t2_ban4","t2_ban5"]]

In [49]:
champs

,t1_champ1id,t1_champ2id,t1_champ3id,t1_champ4id,t1_champ5id,t2_champ1id,t2_champ2id,t2_champ3id,t2_champ4id,t2_champ5id,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,8,432,96,11,112,104,498,122,238,412,92,40,69,119,141,114,67,43,16,51
1,119,39,76,10,35,54,25,120,157,92,51,122,17,498,19,11,67,238,51,420
2,18,141,267,68,38,69,412,126,24,22,117,40,29,16,53,157,238,121,57,28
3,57,63,29,61,36,90,19,412,92,22,238,67,516,114,31,164,18,141,40,51
4,19,29,40,119,134,37,59,141,38,51,90,64,412,25,31,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,19,429,91,432,64,45,83,67,40,79,114,18,11,157,497,55,-1,90,238,157
51486,90,32,31,412,18,102,429,89,75,45,157,164,122,99,420,157,55,119,154,105
51487,141,18,432,81,68,1,119,33,43,99,238,31,24,53,11,113,122,53,11,157
51488,121,236,420,201,99,21,56,25,30,11,119,64,51,157,29,154,39,51,90,114


## 유저가 선택한 캐릭터나 밴한 캐릭터는 데이터에서 제외한다.

In [50]:
data_game.drop(["t1_champ1id","t1_champ2id",
                      "t1_champ3id","t1_champ4id","t1_champ5id","t2_champ1id",
                      "t2_champ2id","t2_champ3id","t2_champ4id","t2_champ5id","t1_ban1","t1_ban2","t1_ban3",
                      "t1_ban4","t1_ban5","t2_ban1","t2_ban2","t2_ban3","t2_ban4","t2_ban5","gameDuration","seasonId"],axis = 1,inplace = True)

#데이터에서 id나 time 필요없이니 제거하기


In [51]:
data_game.drop(["gameId"],axis = 1,inplace = True)
data_game.drop(["creationTime"],axis = 1,inplace = True)


In [52]:
data_game.head()

,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1_sum1,t1_champ1_sum2,t1_champ2_sum1,...,t2_champ3_sum2,t2_champ4_sum1,t2_champ4_sum2,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,1,2,1,1,1,1,2,12,4,3,...,4,14,4,4,3,5,0,0,1,1
1,1,1,1,1,0,1,1,7,4,12,...,4,4,14,4,7,2,0,0,0,0
2,1,2,1,1,1,2,0,4,7,11,...,12,4,11,7,4,2,0,0,1,0
3,1,1,1,1,1,1,0,4,12,4,...,3,4,14,4,7,0,0,0,0,0
4,1,2,1,1,1,1,0,4,12,11,...,4,4,12,4,7,3,0,0,1,0


## 1팀의 총합 킬수와 2팀의 총합 킬수 피쳐를 만들기

In [53]:
data_game["t1_total_kills"] = data_game["t1_champ1_sum1"] + data_game["t1_champ1_sum2"] + data_game["t1_champ2_sum1"] + data_game["t1_champ2_sum2"] + data_game["t1_champ3_sum1"] + data_game["t1_champ3_sum2"] + data_game["t1_champ4_sum1"] + data_game["t1_champ4_sum2"] + data_game["t1_champ5_sum1"] + data_game["t1_champ5_sum2"] 
data_game["t2_total_kills"] = data_game["t2_champ1_sum1"] + data_game["t2_champ1_sum2"] + data_game["t2_champ2_sum1"] + data_game["t2_champ2_sum2"] + data_game["t2_champ3_sum1"] + data_game["t2_champ3_sum2"] + data_game["t2_champ4_sum1"] + data_game["t2_champ4_sum2"] + data_game["t2_champ5_sum1"] + data_game["t2_champ5_sum2"] 
                                                  

In [54]:
data_game.head()

,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1_sum1,t1_champ1_sum2,t1_champ2_sum1,...,t2_champ4_sum2,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t1_total_kills,t2_total_kills
0,1,2,1,1,1,1,2,12,4,3,...,4,4,3,5,0,0,1,1,69,61
1,1,1,1,1,0,1,1,7,4,12,...,14,4,7,2,0,0,0,0,67,78
2,1,2,1,1,1,2,0,4,7,11,...,11,7,4,2,0,0,1,0,65,71
3,1,1,1,1,1,1,0,4,12,4,...,14,4,7,0,0,0,0,0,65,69
4,1,2,1,1,1,1,0,4,12,11,...,12,4,7,3,0,0,1,0,60,65


In [58]:
data_kill = data_game[["t1_total_kills","t2_total_kills","winner"]]
data_kill

,t1_total_kills,t2_total_kills,winner
0,69,61,1
1,67,78,1
2,65,71,1
3,65,69,1
4,60,65,1
...,...,...,...
51485,78,65,2
51486,78,67,2
51487,65,76,2
51488,67,76,2


## 킬 수가 더 많은 팀이 게임에서 승리할까?

In [60]:
data_kill.groupby('winner').mean()

,t1_total_kills,t2_total_kills
winner,,
1,69.102811,68.977681
2,69.016488,69.076536


In [62]:
69.076536 - 69.016488

0.06004800000000898

In [61]:
69.102811	- 68.977681

0.12512999999999863

### 1팀이 이기는 경우 킬수가 평균적으로 약 0.13 높았고 2팀이 이기는 경우에 평균킬수가 약 0.06높았다.

## 따라서 게임에 승패에 대한 총 킬수는 유의미한 차이가 있다고 할 수 없다

## > 억제기를 먼저 부쉬는 팀이 유리할까?

In [63]:
data_game[["firstInhibitor","winner"]].groupby(["firstInhibitor"],as_index = False).mean().sort_values(by = "winner",ascending = False)

,firstInhibitor,winner
2,2,1.913448
0,0,1.488528
1,1,1.091307


여기서 보면 억제기를 2팀이 미는 경우 winner의 평균이 거의 2에 가깝고 억제기를 1팀이 미는 경우 winner의 평균이 거의 1에 가까운것을 확인 할 수 있다.

## 따라서 억제기를 먼저 미는 것과 승리와의 상관관계가 강하다고 할 수 있다.

## 상관계수 그래프도 가능할까??

## 첫번째 킬, 첫번째 용, 첫번째 바론, 첫번째 전령과 승리와의 상관관계도 확인하자